In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv("blind_50s.csv", delimiter=",")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18554 entries, 0 to 18553
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  18554 non-null  int64 
 1   contents    18554 non-null  object
 2   time        18554 non-null  object
 3   keyword     18554 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.9+ KB


In [4]:
df.head()

,Unnamed: 0,contents,time,keyword
0,0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 ...,작성시간\n2020.07.08.,디지털 드로잉
2,2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터...,작성시간\n04.12,디지털 드로잉
4,4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&...,작성시간\n2019.06.12.,디지털 드로잉


In [5]:
list(df["contents"][:5])

['라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...',
 '아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 형 누나들??',
 '손으로 그리는 강의 추천하는 거 있나요?',
 '그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요~?',
 '요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&합정&DMC 쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요? 한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요ㅠㅠ 이번에 D뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요!! 취미로 하시는 분들 계시면 댓글 부탁드려요!^^']

In [6]:
df2 = df[::3]  #3번째 지나면 1개씩 뽑기 (0, 3, 6...)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6185 entries, 0 to 18552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6185 non-null   int64 
 1   contents    6185 non-null   object
 2   time        6185 non-null   object
 3   keyword     6185 non-null   object
dtypes: int64(1), object(3)
memory usage: 193.4+ KB


In [7]:
df2.head()   

,Unnamed: 0,contents,time,keyword
0,0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
3,3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터...,작성시간\n04.12,디지털 드로잉
6,6,갤럭시탭 최신형으로 있어서 다른건 다필요없고 디지털 드로잉좀 배워보고싶어서 아이패드...,작성시간\n10.22,디지털 드로잉
9,9,보통 아이패드 많이 쓰더라구? 나는 갤탭뿐이고... 갤탭으로 한번 해보고싶은데 어디...,작성시간\n10.12,디지털 드로잉
12,12,재밌닷,작성시간\n07.31,드로잉


# 한국어 데이터 전처리 (임의제거 -> 형태소분석,

In [8]:
df = pd.read_csv("blind_50s.csv", delimiter=",")

In [9]:
data = df.drop(columns=['Unnamed: 0'], axis=1)

In [10]:
data.head()

,contents,time,keyword
0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 ...,작성시간\n2020.07.08.,디지털 드로잉
2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터...,작성시간\n04.12,디지털 드로잉
4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&...,작성시간\n2019.06.12.,디지털 드로잉


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18554 entries, 0 to 18553
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   contents  18554 non-null  object
 1   time      18554 non-null  object
 2   keyword   18554 non-null  object
dtypes: object(3)
memory usage: 435.0+ KB


In [12]:
#conents에 해당하는 부분만 'docs'라는 리스트로 만듦

docs = list(data['contents'])

In [13]:
docs

['라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...',
 '아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 형 누나들??',
 '손으로 그리는 강의 추천하는 거 있나요?',
 '그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요~?',
 '요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&합정&DMC 쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요? 한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요ㅠㅠ 이번에 D뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요!! 취미로 하시는 분들 계시면 댓글 부탁드려요!^^',
 '왜 학창시절에보면 남자애들 그림 ㅈㄴ 못그리잖아 글씨도 더럽고 그게난데 그림그리는걸 너무하고싶어 근데 도화지에 그리자신은없어 디지털드로잉 유튜브에 강의많던데 보고하면 많이늘까?',
 '갤럭시탭 최신형으로 있어서 다른건 다필요없고 디지털 드로잉좀 배워보고싶어서 아이패드 살려고하거든 최신형 필요없고 아이팬슬 지원하는거 사면 좋을꺼같은데 어떤걸 사면 좋을까? 아이패드 7세대개 제일 무난할까?',
 '손그림만 그리다가 아이패드 샀는데 어플 추천해줄수있어? 프로크리에이트나 클립스튜디오 많이 쓰는것같은데 디지털 드로잉은 처음이고 포토샵 정도 다룰줄 알아.....ㅋㅋㅋㅋㅋ 유튜브랑 인터넷 이것저것 찾아보는데 엄청 방대하네',
 '아이패드7세대 vs 아이패드 프로 3세대 중에 어떤게 나을까 디지털 드로잉 배워보려고 하는데 초보라 .. 프로는 에바인가 프로는 가격때매.. 7세대는 가격은 괜찮은데 기능 까는 리뷰가 많아서 뭐가 나을지 고민즁 ㅠㅠㅋㅋ',
 '보통 아이패드 많이 쓰더라구? 나는 갤탭뿐이고... 갤탭으로 한번 해보고싶은데 어디서부터 시작해야할지 모르겠어 보통 강의같은걸로 시작하나....? 미대준비했었어서 그림은 

In [14]:
import re
import requests
from bs4 import BeautifulSoup


from konlpy.tag import Okt 
okt = Okt()

In [15]:
# http로 시작되는 url제거
remove_url = [re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", str(content)) for content in docs]

# 숫자 제거
remove_num = [re.sub(r'\d+',' ', str(content)) for content in remove_url]

# 영어 제거 
remove_alpha = [re.sub('[a-zA-Z]' , ' ', str(content)) for content in remove_num]

# Hashtag 제거
remove_tag = [re.sub('[#]+[0-9a-zA-Z_]+', ' ', str(content)) for content in remove_alpha]

# 특수문자 제거
remove_special = [re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣]', ' ', str(content)) for content in remove_tag]

#정규식(Regular Expression)이용해서 정규화
lawdata = [re.sub(r"[^가-힣A-Za-z0-9]", " ", str(content)) for content in remove_special]


   # 쓰레기 단어 제거
   #text = re.sub('[&]+[a-z]+', ' ', text)

   # 특정 문자 제거
   #for i in ['!','@','#','*'] :
       #new_id = new_id.replace(i, "")

   # 띄어쓰기 제거
   #text = text.replace('\n',' ')

In [16]:
lawdata[:20]

['라고 하기엔  개 뿐이지만     배우기 시작한지는 얼마 안되서 아직은 기초단계   ',
 '아이패드    타블렛 뭐가 나을까   클래스   인가 그거 들을만 할까 들어본 형 누나들  ',
 '손으로 그리는 강의 추천하는 거 있나요 ',
 '그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요  ',
 '요즘 반복되는 업무에 치이다보니 취미 좀 정적인것  도전해보고 싶은데요 혹시 일산 합정     쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요  한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요   이번에  뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요   취미로 하시는 분들 계시면 댓글 부탁드려요   ',
 '왜 학창시절에보면 남자애들 그림    못그리잖아 글씨도 더럽고 그게난데 그림그리는걸 너무하고싶어 근데 도화지에 그리자신은없어 디지털드로잉 유튜브에 강의많던데 보고하면 많이늘까 ',
 '갤럭시탭 최신형으로 있어서 다른건 다필요없고 디지털 드로잉좀 배워보고싶어서 아이패드 살려고하거든 최신형 필요없고 아이팬슬 지원하는거 사면 좋을꺼같은데 어떤걸 사면 좋을까  아이패드  세대개 제일 무난할까 ',
 '손그림만 그리다가 아이패드 샀는데 어플 추천해줄수있어  프로크리에이트나 클립스튜디오 많이 쓰는것같은데 디지털 드로잉은 처음이고 포토샵 정도 다룰줄 알아           유튜브랑 인터넷 이것저것 찾아보는데 엄청 방대하네',
 '아이패드 세대    아이패드 프로  세대 중에 어떤게 나을까 디지털 드로잉 배워보려고 하는데 초보라    프로는 에바인가 프로는 가격때매    세대는 가격은 괜찮은데 기능 까는 리뷰가 많아서 뭐가 나을지 고민즁     ',
 '보통 아이패드 많이 쓰더라구  나는 갤탭뿐이고    갤탭으로 한번 해보고싶은데 어디서부터 시작해야할지 모르겠어 보통 강의같은걸로 시작하나      미대준비했었어서 그림은 잘 

In [ ]:
#morphs는 형태소 단위로 구문을 분석(형태소 단위로 토크나이즈)

lawdata_tokened_morphs = []

for doc in lawdata:
    newdoc = okt.morphs(doc)
    lawdata_tokened_morphs.append(newdoc)

In [ ]:
lawdata_tokened_morphs[:2]

In [ ]:
#명사만 추출해줌(토크나이즈해서 리스트 형태로 명사만 추출)

lawdata_tokened_nouns = []

for doc in lawdata:
    newdoc = okt.nouns(doc)
    lawdata_tokened_nouns.append(newdoc)

In [ ]:
lawdata_tokened_nouns[:2]

In [ ]:
#각 품사를 태깅하는 역할. 
#품사를 태깅한다는 것은 주어진 텍스트를 형태소 단위로 나누고, 나눠진 각 형태소를 그에 해당하는 품사와 함께 리스트화 하는 것을 의미한다. 
#형태소 분석으로, 형태소 단위 토크나이징
#pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True)]


pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True) 
                 if ('Noun' in p or 'Adjective' in p or 'Verb' in p or 'Adverb' in p) 
                 and '보다' not in p and '하다' not in p]
lawdata_tokened = [pos(doc) for doc in lawdata]

In [ ]:
lawdata_tokened[:2]

In [ ]:
print(lawdata_tokened[1])
lawdata_tokened[1][0].split('/')

In [ ]:
lawdata_tokened_filtered = []

for doc in lawdata_tokened:
    newdoc = [d for d in doc if d.split('/')[1] == 'Noun' or d.split('/')[1] == 'Adjective' or d.split('/')[1] == 'Adverb']
    lawdata_tokened_filtered.append(newdoc)

In [ ]:
lawdata_tokened_filtered[:2]

In [38]:
#명사만 추출하는 함수 정의(토크나이즈해서 명사만 추출)

import string
def noun_process(text):
    nouns = okt.nouns(text)
    #nouns = [word for word in nouns if len(word) > 1]
    return nouns

In [39]:
print(lawdata[0])
print(noun_process(lawdata[0]))

라고 하기엔  개 뿐이지만     배우기 시작한지는 얼마 안되서 아직은 기초단계   
['개', '뿐', '한지', '얼마', '은', '기초', '단계']


In [40]:
#CountVectorizer: 토큰이 문서별로 몇 번 등장했는지 행렬로 정리해주는 모듈
#CountVectorizer해서 word of bags만들어주는 'bow_transformer' 함수 정의

bow_transformer = CountVectorizer(analyzer=noun_process).fit(lawdata)

In [41]:
#총 단어갯수

len(bow_transformer.vocabulary_)

20125

In [44]:
# 0번째줄 문장의 단어는 어느 위치의 벡터인가?, 몇 번 나오는가?

bow_0 = bow_transformer.transform([lawdata[0]])
print(bow_0)

  (0, 365)	1
  (0, 2152)	1
  (0, 3316)	1
  (0, 8089)	1
  (0, 11405)	1
  (0, 13063)	1
  (0, 19045)	1


In [45]:
# 이제 모든 lawdata를 변형

cv_lawdata= bow_transformer.transform(lawdata)

In [46]:
# CountVectorizer 적용한 lawdata 확인

print('Shape of Sparse Matrix: ', cv_lawdata.shape)
print('Amount of Non-Zero occurrences: ', cv_lawdata.nnz)

Shape of Sparse Matrix:  (18554, 20125)
Amount of Non-Zero occurrences:  267165


In [48]:
# 그런데 여기서 주의해야할 것 하나.
# docs는 현재 document-term matrix 로 구성됨
# NMF를 적용하기 위해서는 term-document matrix가 필요하다.
# 즉, 현재의 Matrix를 transposed matrix(전치행렬)로 바꿔줘야 함.

type(cv_lawdata)

scipy.sparse.csr.csr_matrix

In [49]:
# compressed sparse(희소행렬: 행렬의 값이 대부분 0일때) row matrix인 docs를 transpose 한다. (행열바꾸기)

cv_lawdata_trans = cv_lawdata.transpose()
type(cv_lawdata_trans) 

print('Shape of Sparse Matrix: ', cv_lawdata_trans.shape)
print('Amount of Non-Zero occurrences: ', cv_lawdata_trans.nnz)

Shape of Sparse Matrix:  (20125, 18554)
Amount of Non-Zero occurrences:  267165


# NMF 모델 객체 생성

In [50]:
# NMF 모델 객체 생성
# n_components: 매칭해줄 분류객체(베이킹, 요리..)

from sklearn.decomposition import NMF
model = NMF(n_components=25)

In [51]:
# NMF 모델 학습

W = model.fit_transform(cv_lawdata_trans)

In [55]:
H = model.components_

print(type(H))
print(H.shape)

<class 'numpy.ndarray'>
(25, 18554)


In [56]:
print(H[:,5])

[2.55868269e-02 0.00000000e+00 0.00000000e+00 4.30271201e-03
 3.93195057e-02 2.49370578e-03 4.95836960e-03 3.24745933e-03
 5.19019250e-03 2.13434218e-03 0.00000000e+00 0.00000000e+00
 1.76206212e-02 1.25670666e-02 1.76575933e-04 5.34069045e-05
 1.13411966e-02 7.94193051e-03 1.68847930e-03 1.54705883e-02
 5.27803830e-04 3.98807065e-02 1.70923366e-03 7.30607325e-03
 6.27235200e-03]
